# Assignment Notebook

Loading modules

In [1]:
# System tools
import os

# Data analysis
import pandas as pd
from collections import Counter
from itertools import combinations 
from tqdm import tqdm

# NLP
import spacy
nlp = spacy.load("en_core_web_sm")

# Network analysis tools
import networkx as nx
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)

# Regular expressions 
import re

2022-05-23 19:30:55.430062: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 19:30:55.430097: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Experimenting

In [2]:
# > Tried to load one dataset, but was 
# Get the filepath
#filepath = os.path.join("in", "dblp.txt")
# Load data
#y_test = pd.read_csv(filepath, header=None)

In [3]:
# Get data  
# Get the filepath
filepath = os.path.join("in", "cit-HepPh.txt")
# Reading the filepath 
# data = pd.read_csv(filepath, sep='\t')
with open(filepath, "r") as f:
    txt = f.read()

In [4]:
print(txt[:250])

# Directed graph (each unordered pair of nodes is saved once): Cit-HepPh.txt 
# Paper citation network of Arxiv High Energy Physics category
# Nodes: 34546 Edges: 421578
# FromNodeId	ToNodeId
9907233	9301253
9907233	9504304
9907233	9505235
9907233	95


In [5]:
txt.split("\n", 4)[:4]

['# Directed graph (each unordered pair of nodes is saved once): Cit-HepPh.txt ',
 '# Paper citation network of Arxiv High Energy Physics category',
 '# Nodes: 34546 Edges: 421578',
 '# FromNodeId\tToNodeId']

In [6]:
txt_data = txt.split("\n", 4)[4]

In [7]:
print(txt_data[:100])

9907233	9301253
9907233	9504304
9907233	9505235
9907233	9506257
9907233	9606402
9907233	9607354
9907


In [8]:
outpath = os.path.join("in", "cit_HepPh_read.txt")
with open(outpath, "w") as f:
    f.write(txt_data)

In [9]:
# Get data  
# Get the filepath
filepath = os.path.join("in", "cit_HepPh_read.txt")
# Reading the filepath 
data = pd.read_csv(filepath, header = None, sep='\t', names = ("Source", "Target"))

In [11]:
data.head()

,Source,Target
0,9907233,9301253
1,9907233,9504304
2,9907233,9505235
3,9907233,9506257
4,9907233,9606402


In [13]:
len(data) # Is it the amount of data that is giving me trouble? 

421578

Count edges (directed)

In [2]:
# Create toy data
l = [["A", "B"], ["A", "B"], ["B", "A"], ["C", "A"], ["C", "B"], ["B", "C"], ["A", "C"]]
toy_df = pd.DataFrame(l, columns=["source", "target"])

In [27]:
toy_df.head()

,source,target
0,A,B
1,A,B
2,B,A
3,C,A
4,C,B


In [29]:
# Create toy data
l = [["A", "date_1"], ["B", "date_2"], ["C", "date_3"]]
date_df = pd.DataFrame(l, columns=["source", "date"])

In [32]:
result

,source,target,date
0,A,B,date_1
1,A,B,date_1
2,A,C,date_1
3,B,A,date_2
4,B,C,date_2
5,C,A,date_3
6,C,B,date_3


In [30]:
result = pd.merge(toy_df, date_df, on="source")

In [20]:
toy_df.dtypes

source    object
target    object
dtype: object

In [14]:
count_df = toy_df.apply(pd.value_counts)

In [17]:
count_df["sum"] = count_df.sum(axis=1)

In [21]:
count_df.head()

,source,target,sum
A,3,2,5
B,2,3,5
C,2,2,4


In [24]:
filter_df = toy_df[toy_df["source"].str.contains("A") | toy_df["target"].str.contains("A")]

In [25]:
filter_df

,source,target
0,A,B
1,A,B
2,B,A
3,C,A
6,A,C


In [16]:
toy_df.value_counts().reset_index(name='count')

,source,target,count
0,A,B,2
1,A,C,1
2,B,A,1
3,B,C,1
4,C,A,1
5,C,B,1


In [17]:
data.head()

,Source,Target
0,9907233,9301253
1,9907233,9504304
2,9907233,9505235
3,9907233,9506257
4,9907233,9606402


In [18]:
new_data = data.value_counts().reset_index(name='Weights')

In [19]:
new_data.head()

,Source,Target,Weights
0,1001,9212308,1
1,9701370,9607362,1
2,9701371,9606390,1
3,9701371,9605411,1
4,9701371,9510449,1


In [21]:
new_data["Weights"].unique()

array([1])

I quess it does make sense for there to be no repeat values - it would make no sense for an article to make the same citation twice

In [22]:
new_new_data = data.groupby(['Source','Target']).size().reset_index().rename(columns={0:'count'})

In [23]:
new_new_data.head()

,Source,Target,count
0,1001,9212308,1
1,1001,9305239,1
2,1001,9306240,1
3,1001,9312276,1
4,1001,9312333,1


In [24]:
toy_df.groupby(['source','target']).size().reset_index()

,source,target,0
0,A,B,2
1,A,C,1
2,B,A,1
3,B,C,1
4,C,A,1
5,C,B,1


In [26]:
new_data.insert(1, "Type", "directed")

In [28]:
new_data.head()

,Source,Type,Target,Weights
0,1001,directed,9212308,1
1,9701370,directed,9607362,1
2,9701371,directed,9606390,1
3,9701371,directed,9605411,1
4,9701371,directed,9510449,1


In [29]:
subset_data = new_data[:1800]

Making a graph now I guess

In [35]:
# G = nx.from_pandas_edgelist(subset_data, "Source", "Target", ["Weights"])
G = nx.from_pandas_edgelist(subset_data, source='Source',target='Target', edge_attr=None, create_using=nx.DiGraph())

In [36]:
nx.is_directed(G)

True

In [70]:
pos = nx.spring_layout(G, seed=42)

node_sizes = [3 + 10 * i for i in range(len(G))]
M = G.number_of_edges()
edge_colors = range(2, M + 2)
edge_alphas = [(5 + i) / (M + 4) for i in range(M)]
cmap = plt.cm.plasma

nodes = nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color="indigo")
edges = nx.draw_networkx_edges(
    G,
    pos,
    node_size=node_sizes,
    arrowstyle="->",
    arrowsize=10,
    edge_color=edge_colors,
    edge_cmap=cmap,
    width=2,
)

AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

In [37]:
nx.draw_networkx(G, with_labels=True, node_size=20, font_size=10)

AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

In [73]:
degrees = G.degree()

In [74]:
degrees_df = pd.DataFrame(degrees, columns = ["names", "degrees"])
degrees_df.head()

,names,degrees
0,1001,72
1,9212308,57
2,9701370,17
3,9607362,42
4,9701371,21


In [ ]:
# Betweenness centrality
bc = nx.betweenness_centrality(G)
betweenness_df = pd.DataFrame(bc.items())
betweenness_df.head()

In [56]:
type(new_data["Source"][0])

numpy.int64

In [58]:
new_data['Source'] = new_data['Source'].astype(str)
new_data['Target'] = new_data['Target'].astype(str)